In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

In [ ]:
def rm_main():
    
    papers_dic = {}
    
    url1 = 'https://www.sciencedirect.com/search?pub=Information%20Processing%20%26%20Management&date=2019-2022&accessTypes=openaccess&show=100'
    url2 = 'https://www.sciencedirect.com/search?pub=Pattern%20Recognition&date=2019-2022&articleTypes=REV%2CFLA&accessTypes=openaccess&publicationTitles=272206&show=100'
    url3 = 'https://www.sciencedirect.com/search?pub=Knowledge-Based%20Systems&date=2019-2022&articleTypes=FLA&accessTypes=openaccess&show=100'
    
    papers_dic ['Information Processing and Management'] = url1
    papers_dic ['Pattern Recognition'] = url2
    papers_dic ['knowledge based systems'] = url3

    for key in papers_dic:
        
        url_link = papers_dic[key]
        
        data = [] 

        continue_crawl = True

        while continue_crawl:
            browser.get(url_link)
            time.sleep(5)
            html = browser.page_source
            soup = BeautifulSoup(html, 'lxml')

            continue_crawl = False

            for link in  soup.findAll("a", {"class": "anchor result-list-title-link u-font-serif text-s anchor-default"}):
                paper_link = 'https://www.sciencedirect.com/' + link.get('href')
                browser.get(paper_link)
                time.sleep(2)
                html = browser.page_source
                soup = BeautifulSoup(html, 'lxml')

                paper_details = []

                title = soup.find("span", {"class": "title-text"}).text
                abstract = soup.find("div",{"class":"abstract author"}).find("p").text

                authors_list = []
                for author in soup.findAll("span",{"class":"content"}):
                    try:
                        author_name = author.find("span",{"class":"text given-name"}).text + ' ' + author.find("span",{"class":"text surname"}).text
                        authors_list.append(author_name)
                    except:
                        pass
                authors = ', '.join(authors_list)


                keywords_list = []
                try:
                    keyword_section = soup.findAll('div',{'class':"keywords-section"})

                    if len(keyword_section) == 1:
                        for each in keyword_section[0].findAll('span'):
                            keywords_list.append(each.text)
                    else:
                        for each in soup.findAll('div',{'class':"keywords-section"})[1].findAll('span'):
                            keywords_list.append(each.text)
                except:
                    pass

                keywords = ', '.join(keywords_list)

                year = soup.find("div", {"class": "text-xs"}).text.split()[-2][:-1]
                try:
                    a = int(year)
                except:
                    if year == 'Page':
                        year = soup.find("div", {"class": "text-xs"}).text.split()[4][:-1]
                        try:
                            a = int(year)
                        except:
                            year = soup.find("div", {"class": "text-xs"}).text.split()[5][:-1]
                            try:
                                a = int(year)
                            except:
                                year = soup.find("div", {"class": "text-xs"}).text.split()[-3][:-1]              

                paper_details.extend([title, abstract, authors, keywords, year])
                data.append(paper_details)
                
                break

        df=pd.DataFrame(data, columns=['title', 'abstract', 'authors', 'keywords', 'year'])
        df['journal'] = key
        df.to_csv(key+'.csv')